## Objectives
This case study aims to identify patterns which indicate if a client has difficulty paying their installments which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc. This will ensure that the consumers capable of repaying the loan are not rejected. Identification of such applicants using EDA is the aim of this case study.

 

In other words, the company wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment.

To develop your understanding of the domain, you are advised to independently research a little about risk analytics - understanding the types of variables and their significance should be enough).

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [12]:
# loading application data
df_cust_data= pd.read_csv('Credit_EDA_Case_Study/application_data.csv')
df_cust_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_cust_data.shape

(307511, 122)

In [14]:
# loading previous application data

df_cust_data_prev=pd.read_csv('Credit_EDA_Case_Study/previous_application.csv')

In [15]:
df_cust_data_prev.shape

(1670214, 37)

In [16]:
df_cust_data_prev.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#checking diffrent status for previous applications

df_cust_data_prev.NAME_CONTRACT_STATUS.value_counts()

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64

In [53]:
#reducing the dataframe size as per our analysis

df_cust_data_prev = df_cust_data_prev[['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE',  'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'NAME_CONTRACT_STATUS']]

In [54]:
df_cust_data_prev.shape

(1670214, 6)

In [88]:
#checking current application IDs count for the given status

df_cust_data_prev[['NAME_CONTRACT_STATUS','SK_ID_CURR']].groupby(['NAME_CONTRACT_STATUS']).aggregate('count')

,SK_ID_CURR
NAME_CONTRACT_STATUS,
Approved,1036781
Canceled,316319
Refused,290678
Unused offer,26436


In [119]:
df_cust_data_prev.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_CREDIT,AMT_DOWN_PAYMENT,NAME_CONTRACT_STATUS
0,2030495,271877,Consumer loans,17145.0,0.0,Approved
1,2802425,108129,Cash loans,679671.0,NaN,Approved
2,2523466,122040,Cash loans,136444.5,NaN,Approved
3,2819243,176158,Cash loans,470790.0,NaN,Approved
4,1784265,202054,Cash loans,404055.0,NaN,Refused


In [227]:

# splitting the huge data into 4 dataframes based on status - approved, canceled, refused, unused

df_approved= df_cust_data_prev[(df_cust_data_prev.NAME_CONTRACT_STATUS=='Approved')][['SK_ID_CURR','NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').aggregate('count')
df_canceled = df_cust_data_prev[(df_cust_data_prev.NAME_CONTRACT_STATUS=='Canceled')][['SK_ID_CURR','NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').aggregate('count')
df_refused = df_cust_data_prev[(df_cust_data_prev.NAME_CONTRACT_STATUS=='Refused')][['SK_ID_CURR','NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').aggregate('count')
df_unused_offer = df_cust_data_prev[(df_cust_data_prev.NAME_CONTRACT_STATUS=='Unused offer')][['SK_ID_CURR','NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').aggregate('count')

df_approved.reset_index(inplace=True)
df_approved.columns=['SK_ID_CURR','prev_approved_count']

df_canceled.reset_index(inplace=True)
df_canceled.columns=['SK_ID_CURR','prev_canceled_count']

df_refused.reset_index(inplace=True)
df_refused.columns=['SK_ID_CURR','prev_refused_count']

df_unused_offer.reset_index(inplace=True)
df_unused_offer.columns=['SK_ID_CURR','prev_unused_count']


In [235]:
# joining the customer dataframe to above created dataframes to get the count of previously applied applications
# i.e. count of approved applicaitons, refused applications, unused and canceled applications

df_cust_data = df_cust_data.join(df_approved['prev_approved_count'],how='left',on='SK_ID_CURR')
df_cust_data = df_cust_data.join(df_canceled['prev_canceled_count'],how='left',on='SK_ID_CURR')
df_cust_data = df_cust_data.join(df_refused['prev_refused_count'],how='left',on='SK_ID_CURR')
df_cust_data = df_cust_data.join(df_unused_offer['prev_unused_count'],how='left',on='SK_ID_CURR')

In [236]:
df_cust_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,prev_approved_count,prev_canceled_count,prev_refused_count,prev_unused_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,8.0,2.0,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,9.0,1.0,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,NaN


In [237]:
# filling NaN with 0
df_cust_data.prev_approved_count.fillna(0,inplace=True)
df_cust_data.prev_canceled_count.fillna(0,inplace=True)
df_cust_data.prev_refused_count.fillna(0,inplace=True)
df_cust_data.prev_unused_count.fillna(0,inplace=True)
df_cust_data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,prev_approved_count,prev_canceled_count,prev_refused_count,prev_unused_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,8.0,2.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,9.0,1.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,1.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
